In [ ]:
import pandas as pd
from scipy.stats import binomtest
import matplotlib.pyplot as plt
import plotly.express as px

def transformer(df):
    """
    Diese Funktion verarbeitet ein DataFrame und git ein Array von Dict zurück.
    Arguments:
        df: Ein DataFrame
    Returns:
        [{
          'time': Zeitpunkt der Ziehung.
          'draw1': die erste Zahl der Ziehung.
          'draw2': die zwiete Zahl der Ziehung.
          'draw3': die dritte Zahl der Ziehung.
          'draw4': die vierte Zahl der Ziehung.
          'draw5': die fünfte Zahl der Ziehung.
          'draw6': die sechste Zahl der Ziehung.
          'draw7': die Zusatzzahl der Ziehung.
        },...]
    
    Die Struktur der DataFrame ensteht aus dem Einlesen einer JSON-Datei. Die
    JSON-Daite sieht wie folgt aus:
    
    {
      "div": {
        "h2": "Archiv mit Ziehungen aus dem Jahr 2022",
        "div": [
          {"div": […]},
          {
            "div": [1, …],
            "time": "01.01.2022"
          },
          {
            "div": [2, …],
            "time": "05.01.2022"
          },
        ]
      }
    }
    
    Die Funktion iteriert über die den Array df['div'][0] und verarbeitet nur
    die Zeilen, die ein 'time' als Attribut haben. Diese Zeilen werden in ein
    Dict umgewandelt.
    """
    d = []
    for item in df['div'][0]:
        if 'time' in item:
            d.append({
                'time': pd.to_datetime(item['time'], dayfirst=True),
                'draw1': item['div'][1],
                'draw2': item['div'][2],
                'draw3': item['div'][3],
                'draw4': item['div'][4],
                'draw5': item['div'][5],
                'draw6': item['div'][6],
                'draw7': item['div'][7],
            })
    return d

def convertColumnsInOneColumn(dataFrame):
    """
    Diese Funktion verarbeitet ein DataFrame, der aus dem Array der Funktion
    transformer erstellt wurde.
    Arguments:
        df: Ein DataFrame, siehe die Funktion transformer.
    Returns:
        Die zu einer Spalte zusammengefassten Ziehungen 'draw1', ..., 'draw7'.
    
    Das DataFrame hat die Spalten 'time', 'draw1', ..., 'draw7'. Die Spalten
    'draw1', ..., 'draw7' werden zu einer Spate zusammengefügt. Etwas salopp
    ausgedrückt werden die Spalten, die nebeneinander stehen, mit dieser
    Funktion untereinder geschrieben.
    """
    array = []
    for column in [str(x) for x in list(range(1,7))]:
        array.append(dataFrame["draw" + column])
    return array

def forAllFiles(fromYear, toYear):
    tmp = []
    for year in [str(x) for x in list(range(fromYear, toYear))]:
        div = pd.read_json("json/lotto" + year + ".json")
        transformed = pd.DataFrame(transformer(div))
        tmp.append(transformed)
    return tmp

allData = pd.concat(forAllFiles(1955, 2023))


fig = px.line(allData, x='time', y=['draw1', 'draw2', 'draw3', 'draw4', 'draw5', 'draw6'])

fig.update_layout(xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=3,
                     label="3m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=False
        ),
        type="date"
    ))

In [ ]:
import itertools

def transformer2(df):
    d = []
    for item in df['div'][0]:
        if 'time' in item:
            d.append({
                'time': pd.to_datetime(item['time'], dayfirst=True),
                'draw': [item['div'][1], 
                         item['div'][2], 
                         item['div'][3], 
                         item['div'][4], 
                         item['div'][5], 
                         item['div'][6]],
                'addNumber': item['div'][7]
            })
    return d

df = pd.read_json("json/lotto2022.json")
draws = transformer2(df)
filteredDraws = [item for item in draws if item['time'] > pd.Timestamp(2022, 11, 5)]
filteredDraws

allMyNumbers = [
  {'numbers': [1,  2, 16, 19, 43, 48], 'addNumber': 5},
  {'numbers': [7, 15, 24, 26, 45, 46], 'addNumber': 5},
  {'numbers': [4, 11, 13, 15, 20, 33], 'addNumber': 0},
  {'numbers': [1, 12, 16, 29, 38, 44], 'addNumber': 0}
]

bag = []
for myNumbers in allMyNumbers:
    arrayOfCombinations = []
    for i in range(6, 1, -1):    
        arrayOfCombinations.extend(itertools.combinations(myNumbers['numbers'], i))
    bag.append({'combinations': arrayOfCombinations, 'addNumber': myNumbers['addNumber']})
        
bag

for item in filteredDraws:
    for elements in bag:
        for element in elements['combinations']:
            if(set(element) <= set(item['draw'])):
                print(str(item['time']) + " " + 
                      str(element) + 
                      "[" + str(elements['addNumber']) + "]" + 
                      " in " + str(item['draw']) +
                     "[" + str(item['addNumber']) + "]")
                break

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

%matplotlib notebook

summary = allData.set_index('time')
fig, ax = plt.subplots(2, 1)

@interact(year=(1955, 2022, 1))
def myfilter(year=1955):
    filteredSummary = summary.loc[str(year):str(year)]
    
    overall = pd.concat(convertColumnsInOneColumn(filteredSummary)).value_counts()
    newTable = pd.DataFrame({
        'Number': overall.index,
        'Count': overall.values
    })

    sumOverall = sum(newTable['Count'])
    test = lambda x: binomtest(x, sumOverall, 1/49).pvalue
    newTable['Test'] = newTable['Count'].apply(test)
    ax[0].clear()
    ax[0].bar(newTable.index, newTable['Test'])
    ax[0].plot(newTable.index, [0.01 for x in newTable.index], 'r-')
    
    ax[1].clear()
    ax[1].set_xticks([1,5,10,15,20,25,30,35,40,45,50])
    ax[1].bar(newTable['Number'], newTable['Test'])
    ax[1].plot(newTable['Number'], [0.01 for x in newTable['Number']], 'r-')
    fig.canvas.draw_idle()

# widgets.interact(filter)

In [ ]:
fig2, ax2 = plt.subplots(1, 1)

items=10
maxData = len(allData.index) - items;

@interact(at=(0, maxData, 1), items=10)
def timeLine(at=0, items=10):
    filteredSummary = allData.iloc[at:at + items]
    
    ax2.clear()
    
    plt.rcParams.update({'figure.autolayout': True})
    plt.setp(ax2.get_xticklabels(), rotation=90, horizontalalignment='right')
    ax2.plot(filteredSummary['time'], filteredSummary['draw1'])
    ax2.plot(filteredSummary['time'], filteredSummary['draw2'])
    ax2.plot(filteredSummary['time'], filteredSummary['draw3'])
    ax2.plot(filteredSummary['time'], filteredSummary['draw4'])
    ax2.plot(filteredSummary['time'], filteredSummary['draw5'])
    ax2.plot(filteredSummary['time'], filteredSummary['draw6'])
    
    
    fig2.canvas.draw_idle()

In [ ]:
from scipy.stats import binom
from scipy.stats import norm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

numberOfTrials = 10
probOfSuccess = 0.7

success = {'success': list(range(0, numberOfTrials + 1))}
binomTable = pd.DataFrame(success)
prob = lambda x: binom.pmf(x, numberOfTrials, probOfSuccess)
binomTable['prob'] = binomTable['success'].apply(prob)

mu = numberOfTrials * probOfSuccess
sigma = np.sqrt(numberOfTrials * probOfSuccess * (1 - probOfSuccess))

ticks = {'ticks': np.arange(0, numberOfTrials, 0.1)}
normTable = pd.DataFrame(ticks)
prob = lambda x: norm.pdf(x, mu, sigma)
normTable['prob'] = normTable['ticks'].apply(prob)

plt.figure(figsize=(9,6))
plt.bar(binomTable['success'], binomTable['prob'])
plt.plot(normTable['ticks'], normTable['prob'])

In [ ]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, 2 * np.pi, 200)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line, = ax.plot(x, np.sin(x))

def update(w = 1.0):
    line.set_ydata(np.sin(w * x))
    fig.canvas.draw_idle()

interact(update);

In [ ]:
import plotly as py
import plotly.graph_objects as go
import numpy as np

py.offline.init_notebook_mode(connected=True)

x = np.linspace(0, np.pi, 100)
layout = go.Layout(title='ASDF')
sc = go.Scatter(x=x, y=np.sin(x), mode='lines')

fig = go.Figure([sc])
py.offline.iplot(fig)

In [ ]:
import pandas as pd
import plotly.express as px

eur = pd.read_csv('csv/BBEX3.D.XAU.EUR.EA.AC.C04.csv',
                 sep=";",
                 parse_dates=['time'])

usd = pd.read_csv('csv/BBEX3.D.XAU.USD.EA.AC.C04.csv',
                 sep=";",
                 parse_dates=['time'])

# df = pd.concat([eur, usd])
df = usd.merge(eur, how='left')
# df = usd.join(eur, on='time', how='left')

fig = px.line(df, x='time', y=['valueEUR', 'valueUSD'])

fig.update_layout(xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=3,
                     label="3m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=False
        ),
        type="date"
    ))